<img src="./VSM_logo.gif" width="550" align="center">

## Creation of a Research Object from a single run of VSM with the Reliance (rohub) library aggregating resources from local and from the EGI notebook

Steps 
- Create a RO
- Add metadata - collateral information
- Add resources as internal
- Add resources as external by generating sharable links in the datahub

----

In [1]:
pip install rohub

     |████████████████████████████████| 3.8 MB 4.6 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for rohub: filename=rohub-1.0.7-py3-none-any.whl size=41549 sha256=2f92d1016f4ebb44a9a1420020cdea6372d90776d400bd26938ad70654012aa6
  Stored in directory: /home/jovyan/.cache/pip/wheels/fd/ef/78/531220e1c92c10ef03f672d772cd432d323307c32cdc78534d
Successfully built rohub
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from rohub import rohub, settings
import pathlib
from zipfile import ZipFile
import json
import requests
import datetime
import shutil

### Sign in
### Create a .txt file named "rohub_user.txt" containing the ROHub user name, and a .txt file named "rohub_pwd.txt" reporting the ROHun password

In [15]:
rohub_user = open("/home/jovyan/rohub-user").read().rstrip()
rohub_pwd = open("/home/jovyan/rohub-pwd").read().rstrip()
rohub.login(username=rohub_user, password=rohub_pwd)

Logged successfully as giorgio.castellan@bo.ismar.cnr.it.


### Check my last ROs

In [16]:
myros=rohub.list_my_ros()
myros[0:10]

,identifier,title,description,type,research_areas,status,created_on,created_by,modified_on,access_mode,owner,creation_mode,shared_link
0,6556cdf7-bcef-43d3-a3ce-3d45d14e4a24,Identifying Priorities for the Protection of D...,Benthic habitats of the deep Mediterranean Sea...,Basic Research Object,"[Applied sciences, Ecology, Environmental rese...",LIVE,2022-03-24T17:00:15.895312+00:00,giorgio.castellan@bo.ismar.cnr.it,2022-03-25T09:51:59.432183+00:00,PUBLIC,giorgio.castellan@bo.ismar.cnr.it,MANUAL,https://w3id.org/ro-id/6556cdf7-bcef-43d3-a3ce...
1,f90c2540-1c8c-499b-b3d1-ec95ac526772,Mapping the photic zone of the Mediterranean S...,Estimating the penetration of light along the ...,Basic Research Object,"[Ecology, Environmental research]",LIVE,2022-03-03T16:25:15.908501+00:00,giorgio.castellan@bo.ismar.cnr.it,2022-05-22T16:22:45.282688+00:00,PUBLIC,giorgio.castellan@bo.ismar.cnr.it,MANUAL,https://w3id.org/ro-id/f90c2540-1c8c-499b-b3d1...
2,d0694eaf-a561-4c9f-9a70-17c296da2140,Analysis from satellite data – Environmental m...,Collection and analysis of satellite data to m...,Basic Research Object,[Earth sciences],LIVE,2021-12-14T10:41:17.716553+00:00,giorgio.castellan@bo.ismar.cnr.it,2022-03-29T09:26:43.757264+00:00,PUBLIC,giorgio.castellan@bo.ismar.cnr.it,MANUAL,https://w3id.org/ro-id/d0694eaf-a561-4c9f-9a70...
3,34d648b3-0014-4a19-8469-40b9380ca4c3,Discover and subset satellite data from the AD...,This Research Object demonstrate how to discov...,Executable Research Object,[Earth sciences],LIVE,2021-12-14T09:50:09.833282+00:00,giorgio.castellan@bo.ismar.cnr.it,2022-03-24T17:36:26.806578+00:00,PUBLIC,giorgio.castellan@bo.ismar.cnr.it,MANUAL,https://w3id.org/ro-id/34d648b3-0014-4a19-8469...
4,894d3a33-8340-497d-beaf-5b9d85c9bfc7,Satellite data on water clarity in the Venice ...,Satellite Data on Chlorophyll-a and diffuse at...,Data-centric Research Object,"[Applied sciences, Earth sciences, Optics, Phy...",LIVE,2021-12-07T15:44:04.866966+00:00,giorgio.castellan@bo.ismar.cnr.it,2022-03-24T16:35:02.917450+00:00,PUBLIC,giorgio.castellan@bo.ismar.cnr.it,MANUAL,https://w3id.org/ro-id/894d3a33-8340-497d-beaf...
5,1c87c3d6-46f0-4bfe-bc73-88282fb8c3c5,Spatial and temporal distribution of Cold Wate...,"Data on temperature, salinity, dissolved oxyge...",Data-centric Research Object,"[Applied sciences, Earth sciences, Ecology, Ge...",LIVE,2021-12-07T15:20:52.205188+00:00,giorgio.castellan@bo.ismar.cnr.it,2022-03-24T04:37:09.655295+00:00,PUBLIC,giorgio.castellan@bo.ismar.cnr.it,MANUAL,https://w3id.org/ro-id/1c87c3d6-46f0-4bfe-bc73...
6,f9fca558-8f1d-4aab-a53b-90c4d1bc0037,Mapping the Mediterranean photic zone - Data,Data on light attenuation (Kd490) and Chloroph...,Data-centric Research Object,"[Applied sciences, Earth sciences, Ecology]",LIVE,2021-12-07T14:00:01.195220+00:00,giorgio.castellan@bo.ismar.cnr.it,2022-03-24T04:00:41.708212+00:00,PUBLIC,giorgio.castellan@bo.ismar.cnr.it,MANUAL,https://w3id.org/ro-id/f9fca558-8f1d-4aab-a53b...


### Create a new RO

In [17]:
# ROS_CREATE
# title, research_areas, description=None, access_mode=None, ros_type=None, use_template=False, owner=None, editors=None, readers=None, creation_mode=None

# RO TYPE
#'Basic Research Object', 'Process-centric Research Object', 'Workflow-centric Research Object', 'Bibliography-centric Research Object', 
# 'Executable Research Object', 'Service-centric Research Object', 'Software-centric Research Object', 'Data-centric Research Object'

ro_title="INSERT TITLE"
ro_research_areas=["INSERT RESEARCH AREAS"]
ro_type='INSERT RO TYPE'
#ro.template='Executable Research Object folders structure'
#ro_type="Workflow-centric Research Object"
#ro_template="Workflow-centric Research Object"

ro = rohub.ros_create(title=ro_title, research_areas=ro_research_areas, ros_type=ro_type,use_template=True)

Research Object was successfully created with id = fc8a8087-e49a-4d97-8366-61906b5c3969


### ...or Load a RO

In [86]:
#ro_id=myros.loc[0,"identifier"]
#ro=rohub.ros_load(identifier=ro_id)

### Show metadata

In [18]:
ro.show_metadata()

{'identifier': 'fc8a8087-e49a-4d97-8366-61906b5c3969',
 'shared_link': 'https://w3id.org/ro-id/fc8a8087-e49a-4d97-8366-61906b5c3969',
 'title': 'Demo RO for creating an executable Research Object from a Jupyter Notebook',
 'description': None,
 'status': 'LIVE',
 'access_mode': 'PUBLIC',
 'type': 'Executable Research Object',
 'template': 'Executable Research Object folders structure',
 'created': '2022-07-18T10:23:36.257031Z',
 'creator': 'giorgio.castellan@bo.ismar.cnr.it',
 'modificator': None,
 'modified': '2022-07-18T10:23:36.553703Z',
 'importer': None,
 'research_areas': ['Earth sciences'],
 'owner': 'giorgio.castellan@bo.ismar.cnr.it',
 'editors': [],
 'readers': [],
 'rating': '0.00',
 'number_of_ratings': 0,
 'number_of_likes': 0,
 'number_of_dislikes': 0,
 'completeness': 0,
 'size': 0,
 'creation_mode': 'MANUAL',
 'doi': None,
 'resource_storage': 'DEFAULT',
 'api_link': 'https://api.rohub.org/api/ros/fc8a8087-e49a-4d97-8366-61906b5c3969/'}

### Add description to the RO

In [19]:
ro.description="ADD A DESCRIPTION"

ro.update()

{'identifier': 'fc8a8087-e49a-4d97-8366-61906b5c3969',
 'shared_link': 'https://w3id.org/ro-id/fc8a8087-e49a-4d97-8366-61906b5c3969',
 'title': 'Demo RO for creating an executable Research Object from a Jupyter Notebook',
 'description': 'This Research Object has been created through EGI Jupyter Notebook. It contains the Python script to create, populate and aggreagte resources in a new or an existing RO',
 'status': 'LIVE',
 'access_mode': 'PUBLIC',
 'type': 'Executable Research Object',
 'template': 'Executable Research Object folders structure',
 'created': '2022-07-18T10:23:36.257031Z',
 'creator': 'giorgio.castellan@bo.ismar.cnr.it',
 'modificator': 'giorgio.castellan@bo.ismar.cnr.it',
 'modified': '2022-07-18T10:25:54.960186Z',
 'importer': None,
 'research_areas': ['Earth sciences'],
 'owner': 'giorgio.castellan@bo.ismar.cnr.it',
 'editors': [],
 'readers': [],
 'rating': '0.00',
 'number_of_ratings': 0,
 'number_of_likes': 0,
 'number_of_dislikes': 0,
 'completeness': 0,
 'size

## Add Funder

In [20]:
funded_by = {
"grant_id": "ADD ID",
"grant_Name": "ADD NAME",
"grant_title": "ADD TITLE",
"funder_name": "ADD FUNDER",
"funder_doi": "ADD DOI",
}

if funded_by:
    ro.add_funding(grant_identifier=funded_by["grant_id"], grant_name=funded_by["grant_Name"],
                   funder_name=funded_by["funder_name"], grant_title=funded_by["grant_title"],
                   funder_doi=funded_by["funder_doi"])

## Add license

In [21]:
#licenses = rohub.list_available_licenses()
#licenses
license = 'ADD LICENSE'

ro.set_license(license_id=license) 

{'identifier': 'GPL-3.0-or-later',
 'title': 'GNU General Public License v3.0 or later',
 'description': 'GNU General Public License v3.0 or later',
 'created': '2018-11-22T14:50:40.729699+00:00',
 'updated': '2021-10-07T08:21:24.538648+00:00',
 'creator': None,
 'status': 'active',
 'url': 'https://opensource.org/licenses/GPL-3.0',
 'custom': False}

# Add resources to the RO

### Define folders in datahub

In [91]:
WORKDIR_FOLDER = os.path.join(os.environ['HOME'], "datahub/Reliance/NAME FOLDER")
print("WORKDIR FOLDER: ", WORKDIR_FOLDER)

WORKDIR FOLDER:  /home/jovyan/datahub/Reliance/Geohazard_Santorini2


In [92]:
INPUT_DATA_DIR  = os.path.join(WORKDIR_FOLDER, 'input')
OUTPUT_DATA_DIR = os.path.join(WORKDIR_FOLDER, 'output')
TOOL_DATA_DIR   = os.path.join(WORKDIR_FOLDER, 'tool')

### Add Sketch

In [93]:
resi_res_type = "Sketch"
resi_file_path = os.path.join(OUTPUT_DATA_DIR, 'NAME FILE WITH EXTENSION')
resi_title="ADD TITLE"
resi_folder="output"
resi_description=resi_title

ro.add_internal_resource(res_type=resi_res_type,
                         file_path=resi_file_path,
                         title=resi_title,
                         description=resi_description,
                         folder=resi_folder)

Resource was successfully created with id = 01121b69-cd6e-491e-8f4a-320ad9ec4784


Resource(identifier=01121b69-cd6e-491e-8f4a-320ad9ec4784, post_request=False)

### or from external resource

In [ ]:
resi_res_type = "Sketch"
resi_file_url = "https://www.researchobject.org/images/research-objects-illustration.png"
resi_title="Research Object Conceptualization"
resi_folder="output"
resi_description=resi_title

ro.add_external_resource(res_type=resi_res_type,
                         input_url=resi_file_url,
                         title=resi_title,
                         description = resi_description,
                         folder=resi_folder)

### Add bibliographic resource

In [95]:
rese_res_type="Bibliographic Resource"
rese_file_url="ADD URL TO RESOURCE"
rese_title="ADD RESOURCE TITLE"
rese_descr= "ADD DESCRIPTION / REFERENCE"
resi_folder="biblio"

ro.add_external_resource(res_type=rese_res_type,
                         input_url=rese_file_url,
                         title=rese_title,
                         description = rese_descr,
                         folder=resi_folder)

Resource was successfully created with id = 4011d8da-4641-4ad5-83e6-0a6f43b263e1


Resource(identifier=4011d8da-4641-4ad5-83e6-0a6f43b263e1, post_request=False)

In [96]:
ro.list_resources()

,identifier,type,source,title,description,url,name,path,size,creator,created_on,modified_on,download_url
0,4011d8da-4641-4ad5-83e6-0a6f43b263e1,Bibliographic Resource,external,Monitoring Santorini volcano (Greece) breathin...,Monitoring Santorini volcano (Greece) breathin...,https://academic.oup.com/gji/article/193/1/161...,Monitoring Santorini volcano (Greece) breathin...,https://academic.oup.com/gji/article/193/1/161...,NaN,rpalma@man.poznan.pl,2022-06-22T20:33:11.784269+0000,2022-06-22T20:33:12.290539+0000,https://api.rohub.org/api/resources/4011d8da-4...
1,01121b69-cd6e-491e-8f4a-320ad9ec4784,Sketch,internal,Data - Model - Residuals with InSAR descending...,Data - Model - Residuals with InSAR descending...,None,Data - Model - Residuals with InSAR descending...,output/VSM_res_sar.png,126137.0,rpalma@man.poznan.pl,2022-06-22T20:33:03.995312+0000,2022-06-22T20:33:08.357713+0000,https://api.rohub.org/api/resources/01121b69-c...


### Add Python Script

In [ ]:
import pathlib
resi_file_path='DEMO_ROcreator.ipynb'
resi_folder=myfolders.loc[3,"identifier"]

In [ ]:
if pathlib.Path(resi_file_path).exists():
    resi_res_type="Python Script"
    resi_title="Create a new RO using Python Jupyter Notebook through EGI"
    resi_description= resi_title
    ro.add_internal_resource(res_type=resi_res_type,file_path=resi_file_path, title=resi_title, description=resi_description)

# Aggregate external resources from the EGI datahub

###  Get shared link from datahub
1. Retrieve your client identifier, client secret and refresh token from https://aai.egi.eu/fedcloud/
2. Create a new file in your HOME area for instance using nano (keep the `$` character in front of HOME; this is meant to be an environment variable):
```
nano $HOME/egi_fedcloud.cfg
```
3. Paste your client identifier, client secret and refresh token using the following syntax (do not forget `{` and `}` as well as comma `,` and columns `:`
```
{
"id": "XXXXXXXXXXXXXXXXX",
"secret": "YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY",
"token": "ffffffffffffffffffffffff"
}
```


4a.NOTE: New refresh tokens expire in 13 months.
To generate access tokens from this refresh token use the following curl command:
curl -X POST -u 'f33e824a-078d-497b-b700-25b0df7fc5b7':'B80vPK0LVbYuvwRj0Aexs8y0rKgk5XHwYRRq3BCr33ejj33385bzDVcPmSTUkqA2QjMiwWKJDTxvOou7yVV8EA'  -d 'grant_type=refresh_token&refresh_token=eyJhbGciOiJub25lIn0.eyJleHAiOjE2Nzg3MjI0MTgsImp0aSI6Ijg5NDg5NTYwLTJkOWEtNGQ3OS05M2IwLWQ4ZDAxNTA3NWVmNiJ9.&scope=openid%20email%20profile%20eduperson_entitlement' 'https://aai.egi.eu/oidc/token' | python -m json.tool;
NOTE: New access tokens expire in 1 hour.
You can manage your refresh tokens in the following link: https://aai.egi.eu/oidc/manage/user/services

4b. You are ready to go. If you have issues, check first that the syntax is correct in your egi_fedcloud.cfg, that this file is located in the correct folder and then finally that your refresh token is still valid (check on https://aai.egi.eu/fedcloud/)

#### EGI Datahub functions to initialize EGI datahub and get shared link

In [98]:
def egi_datahub_init():
    OIDC_ENDPOINT="https://aai.egi.eu/oidc/"
    ONEZONE_ENDPOINT="https://datahub.egi.eu/api/v3/onezone/"
    ONEPROVIDER_ENDPOINT="https://cesnet-oneprovider-01.datahub.egi.eu/api/v3/oneprovider/"

    egi_fedcloud_filename = os.path.join(os.environ['HOME'], 'egi_fedcloud.cfg')
    with open(egi_fedcloud_filename, 'r') as convert_file:
        tmp = convert_file.read()
        egi_fedcloud_auth = json.loads(tmp)
        try:
            # Retrieving an OIDC token from Check-in
            data = {
                'client_id': egi_fedcloud_auth['id'],
                'client_secret': egi_fedcloud_auth['secret'],
                'refresh_token': egi_fedcloud_auth['token'],
                #'scope': 'openid email profile',
                'scope': 'openid email profile eduperson_entitlement',
                'grant_type': 'refresh_token'
            }
            response = requests.post(OIDC_ENDPOINT+'token', data=data, auth=(egi_fedcloud_auth['id'], 
                                                                                       egi_fedcloud_auth['secret']))
            #print(json.dumps(response.json(), indent=2))
            EGItoken = response.json()['access_token']
            headers = {
                'X-Auth-Token': f"egi:" + EGItoken,
                'Content-type': 'application/json',
            }
            # get current timestamp
            ts = datetime.datetime.now().timestamp()
            data = json.dumps({ 
                'name': 'REST and CDMI access token ' + str(ts), 
                'type': { 
                    'accessToken': {} 
                }, 
                'caveats': [ { 
                    'type': 'interface', 
                    'interface': 'rest' 
                }] 
            })

            response = requests.post(ONEZONE_ENDPOINT+'user/tokens/named', headers=headers, data=data)
            DATAHUB_TOKEN = response.json()['token']
            return DATAHUB_TOKEN
        except:
            print("EGI Datahub Authentication problem: check your credentials")

In [99]:
def egi_datahub_getlink(datahub_token, filename):
    OIDC_ENDPOINT="https://aai.egi.eu/oidc/"
    ONEZONE_ENDPOINT="https://datahub.egi.eu/api/v3/onezone/"
    ONEPROVIDER_ENDPOINT="https://cesnet-oneprovider-01.datahub.egi.eu/api/v3/oneprovider/"
    
    bname = os.path.basename(filename)
    hname = filename.split('datahub/')[1]
    ## get file id
    headers = { 
        'X-Auth-Token': datahub_token 
    }
    response = requests.post(ONEPROVIDER_ENDPOINT+'lookup-file-id/'+hname, headers=headers)
    dh_fileid = response.json()['fileId']
    ## get shared link 
    headers = { 'X-Auth-Token': datahub_token, 'Content-Type': 'application/json',}
    data = json.dumps({ 
        'name': bname,
        'fileId': dh_fileid
    })
    response = requests.post(ONEPROVIDER_ENDPOINT+'shares', headers=headers, data=data)
    shareIdGenerated=response.json()['shareId']
    # get shared link details
    headers = {
        'X-Auth-Token': datahub_token
    }
    response = requests.get(ONEPROVIDER_ENDPOINT+'shares/'+shareIdGenerated, headers=headers)
    publicURL=response.json()['publicUrl'] #publicURL is nice link but to webpage where you can download file, not the actual file
    contentURL= ONEZONE_ENDPOINT+'shares/data/'+response.json()['rootFileId']+'/content' #use this link to aggregate Jupyter Notebooks
    return (publicURL, contentURL)

### EGI DataHub initialization 

In [100]:
DATAHUB_TOKEN = egi_datahub_init()

## Tool

In [107]:
myfilename = 'ADD NAME NOTEBOOK'
shared_res_path = os.path.join(TOOL_DATA_DIR, myfilename)
print(shared_res_path)


/home/jovyan/datahub/Reliance/Geohazard_Santorini2/tool/VSM_Santo_datahub.ipynb


In [108]:
links = egi_datahub_getlink(DATAHUB_TOKEN, shared_res_path)
res_file_url = links[1]

print (res_file_url)

https://datahub.egi.eu/api/v3/onezone/shares/data/00000000007EA412736861726547756964236263666465323337623438616464356432313236613730653731643138646262636830373531233732356634616233366362323664306662666330633132346337373565666565636865653439236162643837393636323763613933333536646263343432623734623263616436636831363030/content


In [109]:
#res_file_url = egi_datahub_getlink(DATAHUB_TOKEN, shared_res_path)
res_type="Jupyter Notebook"
res_title="ADD TITLE"
res_description="ADD DESCRIPTION"
res_folder='tool'
ro.add_external_resource(res_type=res_type,
                         input_url=res_file_url,
                         title=res_title, 
                         description=res_description, 
                         folder=res_folder)

Resource was successfully created with id = cd79ed15-5aa1-4548-9e06-730095fea961


Resource(identifier=cd79ed15-5aa1-4548-9e06-730095fea961, post_request=False)